<a href="https://colab.research.google.com/github/GarnetKangSB/MultiCampus_2020NLP_BOK_3/blob/master/tone_mkt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np

In [ ]:
minutes=pd.read_csv("./minutes/minutes_ngram_label.csv")
news=pd.read_csv("./minutes/news_ngram_label.csv")
report=pd.read_csv("./minutes/reports_label.csv")

In [ ]:
def preprocess_ngram(string):
    return ",".join([x.split("'")[1] for x in string.split(",")])

In [ ]:
minutes["ngram"]=minutes["ngram"].str.replace("@","")
news["ngram"]=news["ngram"].str.strip("[]")
news["ngram"]=news["ngram"].apply(preprocess_ngram)
report["ngram"]=report["ngram"].str.strip("[]")
report=report.drop(649)
report["ngram"]=report["ngram"].apply(preprocess_ngram)

In [ ]:
concate_data=pd.concat([minutes,news,report])
concate_data=concate_data.loc[(~(concate_data["label"]=="no_change"))&(concate_data["ngram"].notnull())]
concate_data["label"]=concate_data["label"].apply(lambda x: 1 if x=="hawkish" else 0)
concate_data

,Unnamed: 0,date,ngram,label
0,0.0,2005-05-12,"내외/NNG,금리갭/NNG,역전/NNG,자금/NNG,해외/NNG,유출/NNG,대규모...",1
2,2.0,2005-07-07,"들/VV,국제/NNG,유가/NNG,크/VA,오르/VV,원화/NNG,엔/NNG,화간/...",0
3,3.0,2005-08-11,"위안/NNG,절상/NNG,원화/NNG,절상/NNG,가장/MAG,크/VA,추가/NNG...",1
5,5.0,2005-10-11,"금리갭/NNG,확대/NNG,되/XSV,가운데/NNG,남/VV,fed/NNG,금리/N...",1
7,7.0,2005-12-08,"들/VV,달러/NNG,엔/NNG,유로/NNG,대해/VV,상당히/MAG,강세/NNG,...",1
...,...,...,...,...
3555,NaN,2020-01-06,"fed/NNG,다시/MAG,긴축/NNG,길/NNG,금리/NNG,동결/NNG,fed/...",1
3567,NaN,2020-01-28,"크레딧/NNG,투자전략/NNG,신용/NNG,분석/NNG,민정/NNG,회사채/NNG,...",0
3568,NaN,2020-01-28,"크레딧/NNG,이슈/NNG,신용스프레드/NNG,동향/NNG,전망/NNG,발행시장/N...",0
3569,NaN,2020-01-28,"고지/NNG,사항/NNG,조사/NNG,분석/NNG,자료/NNG,신뢰/NNG,하/VV...",0


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
def split(string):
    return string.split(",")

In [ ]:
from collections import defaultdict

In [ ]:
polarity_score_ls=defaultdict(list)
for i in range(30):
    X_train,_,y_train,_= train_test_split(concate_data["ngram"],concate_data["label"],test_size=0.1,stratify=concate_data["label"],shuffle=True)
    
    vectorizer=CountVectorizer(tokenizer=split,lowercase=False)
    ngram_vec=vectorizer.fit_transform(X_train)
    ngram_dict=vectorizer.vocabulary_
    
    model=MultinomialNB()
    model.fit(ngram_vec,y_train)
    likelyhood_prop=np.exp(model.feature_log_prob_)
    
    for ngram in ngram_dict.keys():
        polarity_score_ls[ngram].append((likelyhood_prop[1]/likelyhood_prop[0])[ngram_dict[ngram]])

polarity_score_ls

defaultdict(list,
            {'뉴욕/NNG': [1.1459279783274607,
              1.157726301385147,
              1.152657477108727,
              1.1689720192894,
              1.1422465389164964,
              1.1536392089146077,
              1.1622729155258116,
              1.1418657032456654,
              1.1513881101643868,
              1.1570231545985254,
              1.1547745863177983,
              1.1585668268640807,
              1.1574076076244089,
              1.1640313209999302,
              1.1603978834841062,
              1.1495828076254362,
              1.1568765429430972,
              1.1649541947176252,
              1.1577633733328174,
              1.1559817271767419,
              1.1601424682023223,
              1.1699785892961614,
              1.1637029551493652,
              1.1529516948905167,
              1.1536872903257491,
              1.1511138327034482,
              1.1516493441212519,
              1.1513717361391897,
              1.162631540

In [ ]:
ngram_score=pd.DataFrame({"ngram":list(polarity_score_ls.keys()),"polarity_score":[sum(x)/len(x) for x in polarity_score_ls.values()]})
ngram_score.head(5)

,ngram,polarity_score
0,뉴욕/NNG,1.156101
1,금융시장/NNG,0.823847
2,금리/NNG,1.016991
3,인하/NNG,0.370627
4,기대/NNG,0.883149


In [ ]:
hawkish_ngram_ls=ngram_score.loc[ngram_score["polarity_score"]>1.3,"ngram"].tolist()
dovish_ngram_ls=ngram_score.loc[ngram_score["polarity_score"]<(1/1.3),"ngram"].tolist()

In [ ]:
sentence2=pd.read_csv("./minutes/sentences_ngram_sec2.csv")
sentence3=pd.read_csv("./minutes/sentences_ngram_sec3.csv")

In [ ]:
sentence_data=pd.concat([sentence2,sentence3]).sort_values(by="date")
sentence_data["ngram"]=sentence_data["ngram"].str.strip("[]")
sentence_data["ngram"]=sentence_data["ngram"].apply(preprocess_ngram)
sentence_data = sentence_data.reset_index()
sentence_data=sentence_data[["sentences","date","ngram"]]
sentence_data.head(5)

,index,Unnamed: 0,Unnamed: 1,sentences,date,ngram
0,0,0,0,일부 위원은 내외금리차 역전에 따라 자금의 해외유출이 대규모로 발생될 가능성을 상당...,20050512,"내외/NNG,금리갭/NNG,역전/NNG,자금/NNG,해외/NNG,유출/NNG,대규모..."
1,6,0,6,또 다른 일부 위원은 현재 주택담보대출의 대부분이 변동금리부 조건으로 이루어지고 있...,20050512,"또/MAG,주택담보대출/NNG,변동금리부/NNG,조건/NNG,이루/VV,점/NNG,..."
2,0,0,0,일부 위원은 4월중 연기금의 주식매매 패턴이 주식시장의 안정과 역행하는 방향으로 작...,20050512,"중/NNG,연기금/NNG,주식/NNG,매매패턴/NNG,주식시장/NNG,안정/NNG,..."
3,3,0,3,다른 일부 위원은 최근 은행들의 주택담보대출 증대 노력은 과거와 같은 신규대출 확대...,20050512,"은행/NNG,주택담보대출/NNG,증대/NNG,노력/NNG,같/VA,신규/NNG,대출..."
4,1,0,1,동 위원은 최근 은행들이 마땅한 자금 운용대상 확보가 쉽지 않음에 따라 주택담보대출...,20050512,"은행/NNG,마땅/NNG,자금/NNG,운용/NNG,대상/NNG,확보/NNG,쉽/VA..."


In [ ]:
def sen_tone(ls):
    return len([x for x in ls if x in hawkish_ngram_ls])-len([x for x in ls if x in dovish_ngram_ls])

In [ ]:
sentence_data["sen_tone"]=sentence_data["ngram"].str.split(",").apply(sen_tone)

In [ ]:
hawkish_num=sentence_data.loc[sentence_data["sen_tone"]>0].pivot_table(index="date",values=["sentences"],aggfunc="count")
dovish_num=sentence_data.loc[sentence_data["sen_tone"]<0].pivot_table(index="date",values=["sentences"],aggfunc="count")

In [ ]:
haw_dov_sen_tone=hawkish_num.merge(dovish_num,how="outer",left_index=True,right_index=True).fillna(0)
haw_dov_sen_tone.columns=["hawk_num","dov_num"]
haw_dov_sen_tone

,hawk_num,dov_num
date,,
20050512,4.0,7.0
20050609,5.0,4.0
20050707,4.0,3.0
20050811,13.0,3.0
20050908,10.0,5.0
...,...,...
20190228,9.0,27.0
20190418,14.0,12.0
20190531,13.0,13.0


In [ ]:
tone_doc=(haw_dov_sen_tone["hawk_num"]-haw_dov_sen_tone["dov_num"])/(haw_dov_sen_tone["hawk_num"]+haw_dov_sen_tone["dov_num"])
tone_doc

date
20050512   -0.272727
20050609    0.111111
20050707    0.142857
20050811    0.625000
20050908    0.333333
              ...   
20190228   -0.500000
20190418    0.076923
20190531    0.000000
20190718   -0.404255
20190830   -0.545455
Length: 161, dtype: float64

In [ ]:
tone_doc.to_csv("./minutes/tone_doc.csv")